In [2]:
!pip install -q --upgrade pip
!pip install -q -U accelerate peft bitsandbytes transformers trl einops

In [3]:
# Install and import the necessary libraries
# !pip install torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate einops 

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [4]:
# Model
base_model = "microsoft/phi-2"
# new_model = "phi-2-medquad"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset, DatasetDict, Dataset
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a function to transform the dataset
def format_example(example):
    # Use the specified format for text
    example["text"] = f"### Instruction: {example['Question']} ### Assistant: {example['Answer']}"
    return example

train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset=train_test_split["train"]
train_dataset = train_dataset
test_dataset=train_test_split["test"]
test_dataset = test_dataset

# Apply the transformation to both the train and test datasets
train_dataset = train_dataset.map(format_example)

# Remove unnecessary columns and keep only the "text" column
train_dataset = train_dataset.remove_columns(["qtype", "Question", "Answer"])
# print(formatted_dataset)

# Preview the dataset
print(train_dataset)
print(test_dataset)

README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

Map:   0%|          | 0/13125 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 13125
})
Dataset({
    features: ['qtype', 'Question', 'Answer'],
    num_rows: 3282
})


In [6]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map='auto',
    revision="refs/pr/23" #the main version of Phi-2 doesn’t support gradient checkpointing (while training this model)
)

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

PhiModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [8]:
from peft import PeftModel
f_model = PeftModel.from_pretrained(model,'/kaggle/input/prerak_ka_model/transformers/default/1/phi_subjective_finetuned_model/phi_subjective_finetuned_model')
f_model = f_model.merge_and_unload()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [9]:
from tqdm import tqdm

f_model.eval()  # Set the model to evaluation mode
predictions = []
all_preds = []  # Ensure this is defined for appending predictions
device = next(f_model.parameters()).device  # Ensure device compatibility
batch_size = 1  # Set a batch size if needed

for i, question in enumerate(tqdm(test_dataset, desc="Generating predictions", unit="question")):
    prompt = question['Question']

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = f_model.generate(inputs['input_ids'], max_length=70)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(response)

Generating predictions: 100%|██████████| 3282/3282 [3:43:06<00:00,  4.08s/question]  


In [10]:
import csv

# Open the CSV file in write mode
with open("output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    
    # Optional: Write a header if needed
    writer.writerow(["Prediction"])

    # Write each item from the predictions list
    for item in predictions:
        writer.writerow([item])

with open("true_output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    
    # Optional: Write a header if needed
    writer.writerow(["Prediction"])

    # Write each item from the predictions list
    for item in test_dataset['Answer']:
        writer.writerow([item])

In [11]:
!pip -q install evaluate
!pip -q install rouge-score
# Load ROUGE for evaluation
import evaluate

# Load ROUGE for evaluation
rouge = evaluate.load("rouge")

# Evaluate predictions


# Prepare references (ground-truth answers)
references = test_dataset['Answer']

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
scores = rouge.compute(predictions=predictions, references=references)
print(scores)

{'rouge1': 0.23966662675106465, 'rouge2': 0.09147918662705809, 'rougeL': 0.17345379357463553, 'rougeLsum': 0.18020977495126683}


In [13]:
bleu = evaluate.load("bleu")
scores = bleu.compute(predictions=predictions, references=references)
print(scores)

{'bleu': 0.005795848399947969, 'precisions': [0.4886992174634002, 0.17757447399486148, 0.08904618651566094, 0.051904383812136086], 'brevity_penalty': 0.040954971771938666, 'length_ratio': 0.23836299548976397, 'translation_length': 176094, 'reference_length': 738764}


In [ ]:
meteor = evaluate.load("meteor")
scores = meteor.compute(predictions=predictions, references=references)
print(scores)

In [ ]:
!pip -q install bert_score
from bert_score import score

P, R, F1 = score(predictions, references, lang="en")
print(f"BERTScore F1: {F1.mean().item()}")